In [7]:
from bs4 import BeautifulSoup as soup 
from urllib.request import urlopen as uReq
from urllib.request import Request
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import os
import re

In [8]:
myurl =  'https://cedh-decklist-database.com'
cards = [] # initializes list of all cards

In [45]:

client = uReq(myurl) # opening connection 
page_html = client.read() # content to variable
client.close() # closes connection 
page_soup = soup(page_html, 'html.parser') # html parsing 
containers = page_soup.findAll("div",{"class": "ddb-section"})
htmls = []
for c in containers:
    if c.get_text().strip() == 'COMPETITIVE':
        x = c.parent.parent.find("ul", {"class": "ddb-decklists"})
        htmls.append(x.a["href"])
htmls

['https://www.moxfield.com/decks/hbLMs7xcfUyun2nxK-sRlw',
 'https://www.moxfield.com/decks/1t0Cpm1xqUeyA-6xi2TrGQ',
 'https://www.moxfield.com/decks/KtYYMEkhuUeFdC-LerpWEA',
 'https://www.moxfield.com/decks/u8vKYsL9mkWTItI1JCR85Q',
 'https://www.moxfield.com/decks/SMA-mb153Uqu1NPKPZJ0OA',
 'https://www.mtggoldfish.com/deck/3982753#paper',
 'https://www.mtggoldfish.com/deck/3982746#paper',
 'https://www.moxfield.com/decks/IkyDwh_o0UWLzXROMWSlhA',
 'https://www.moxfield.com/decks/qTHHbx2Zd0el3GaJ09niUg/primer',
 'https://www.moxfield.com/decks/ThcETLXsHk27KhZqxQ1cSg',
 'https://www.moxfield.com/decks/__lFQa0Nt0ymhyYcw8v7kg',
 'https://www.moxfield.com/decks/_4HeABv9DEeykmnZy1JICg',
 'https://www.moxfield.com/decks/651y_CvPcUCmD-J5bd_VYQ',
 'https://www.moxfield.com/decks/W0tbKZGNyEa5vXx9JBOxNA',
 'https://www.moxfield.com/decks/2QnGQIegsUaChGQe31VvJQ',
 'https://www.moxfield.com/decks/VfAj83sObUaqAn9KJqBq_g',
 'https://www.moxfield.com/decks/UlP4pj3SyU2ytH86lnRH5Q',
 'https://www.moxfiel

In [49]:
#cleaning moxfield primer lists
htmls = list(map(lambda x: x.split('/primer')[0],htmls))
htmls

['https://www.moxfield.com/decks/hbLMs7xcfUyun2nxK-sRlw',
 'https://www.moxfield.com/decks/1t0Cpm1xqUeyA-6xi2TrGQ',
 'https://www.moxfield.com/decks/KtYYMEkhuUeFdC-LerpWEA',
 'https://www.moxfield.com/decks/u8vKYsL9mkWTItI1JCR85Q',
 'https://www.moxfield.com/decks/SMA-mb153Uqu1NPKPZJ0OA',
 'https://www.mtggoldfish.com/deck/3982753#paper',
 'https://www.mtggoldfish.com/deck/3982746#paper',
 'https://www.moxfield.com/decks/IkyDwh_o0UWLzXROMWSlhA',
 'https://www.moxfield.com/decks/qTHHbx2Zd0el3GaJ09niUg',
 'https://www.moxfield.com/decks/ThcETLXsHk27KhZqxQ1cSg',
 'https://www.moxfield.com/decks/__lFQa0Nt0ymhyYcw8v7kg',
 'https://www.moxfield.com/decks/_4HeABv9DEeykmnZy1JICg',
 'https://www.moxfield.com/decks/651y_CvPcUCmD-J5bd_VYQ',
 'https://www.moxfield.com/decks/W0tbKZGNyEa5vXx9JBOxNA',
 'https://www.moxfield.com/decks/2QnGQIegsUaChGQe31VvJQ',
 'https://www.moxfield.com/decks/VfAj83sObUaqAn9KJqBq_g',
 'https://www.moxfield.com/decks/UlP4pj3SyU2ytH86lnRH5Q',
 'https://www.moxfield.com/d

In [46]:
chromedriver_path=os.path.join(os.getcwd(), "chromedriver.exe")
options = Options()
options.set_headless(headless=True)
for decklist in htmls:
    if 'moxfield' in decklist: # checks if its a moxfield deck
        driver = webdriver.Chrome(chromedriver_path, options= options)
        driver.get(decklist)
        time.sleep(3) #if you want to wait 3 seconds for the page to load
        page_source = driver.page_source
        driver.close()
        page_soup = soup(page_source, 'html.parser')
        containers = page_soup.findAll("tr",{"class": "table-deck-row"})
        for c in containers:
            cards.append( c.a.get_text())
        cards = cards[:len(cards)-8]
    # if 'tappedout' in decklist:#checks if its a tapped out deck
    #    driver = webdriver.Chrome(chromedriver_path, options= options)
    #    driver.get(decklist)
    #    time.sleep(3) #if you want to wait 3 seconds for the page to load
    #    page_source = driver.page_source
    #    driver.close()
    #    page_soup = soup(page_source, 'html.parser')
    #    containers = page_soup.findAll("li",{"class": "member"})
    #    for c in containers:
    #       if '\n' in c.span.a.get_text():
    #           card = c.span.a.get_text()[16:].replace('\n','').strip()
    #           cards.append(card)

In [90]:
card_data = pd.DataFrame({'Card Name': cards}) # creates a data frame of all cards


In [91]:
card_vc = pd.DataFrame(card_data.value_counts())
card_vc.rename( columns={0 :'Occurrences'}, inplace=True ) # sets coorrect column names
card_vc = card_vc.reset_index()

In [95]:
card_vc.to_json('results/competitiveCards.json', orient='records')
card_vc.to_csv('results/competitive_cards.csv')

In [96]:
my_collection = pd.read_csv('ArchidektCollection-Sat Jul 10 2021.csv')
my_collection.head()

,Quantity,Name
0,1,"Tasigur, the Golden Fang"
1,1,Witherbloom Apprentice
2,1,Emiel the Blessed
3,1,"Valentin, Dean of the Vein // Lisette, Dean of..."
4,1,Goblin Matron


In [97]:
sd = pd.merge(card_vc,my_collection, how='left', left_on='Card Name', right_on='Name', indicator=True)
sd[sd['_merge']=='left_only'].head(40)

,Card Name,Occurences,Quantity,Name,_merge
34,Imperial Seal,193,NaN,NaN,left_only
52,Timetwister,144,NaN,NaN,left_only
61,Wheel of Fortune,137,NaN,NaN,left_only
82,Volcanic Island,110,NaN,NaN,left_only
90,Badlands,106,NaN,NaN,left_only
93,Necropotence,102,NaN,NaN,left_only
96,Gaea's Cradle,99,NaN,NaN,left_only
121,Cavern of Souls,79,NaN,NaN,left_only
130,Savannah,75,NaN,NaN,left_only
135,Taiga,72,NaN,NaN,left_only
